# Gradient-based planner

This notebook demonstrates how to build a gradient-based planner.

In [ ]:
import os

# These environment variables control where training and eval logs are written.
# You can set these in your shell profile as well.
os.environ["RUN_DIR"] = "runs"
os.environ["EVAL_RUN_DIR"] = "eval_runs"
os.environ["MODEL_DIR"] = "models"
os.environ["DATA_DIR"] = "data"

# This is used to set a constant Tensorboard port.
os.environ["TENSORBOARD_PORT"] = str(8989)

import ml.api as ml  # Source: https://github.com/codekansas/ml-starter

# Enables logging to `stdout`.
ml.configure_logging(use_tqdm=True)

# Imports these files to add them to the model and task registry.
from usa.models.point2emb import Point2EmbModel
from usa.tasks.clip_sdf import ClipSdfTask

## Training a model

For this example, we use a clip recorded using the code in the `home-robot` repository [here](https://github.com/facebookresearch/home-robot). You can record your own clip on the Stretch robot and use that instead, by substituting the dataset path.

In [ ]:
import requests
from pathlib import Path
from omegaconf import OmegaConf

data_root = Path("data")
data_root.mkdir(exist_ok=True)
dataset_path = data_root / "dataset.pkl"

# We're downloading an existing dataset, but you can use your own instead.
dataset_url = "https://github.com/codekansas/usa/releases/download/v0.0.2/chris_lab.pkl"
if not dataset_path.exists():
    with requests.get(dataset_url, stream=True) as r:
        r.raise_for_status()
        with open(dataset_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Using the default config, but overriding the dataset.
config = OmegaConf.load("config.yaml")
config.task.dataset = "home_robot"
config.task.dataset_path = str(dataset_path)

# We're using a small number of training steps to make the example easier
# to follow, but this can be configured to improve the model quality.
config.task.finished.max_steps = 500

# We also only use the Tensorboard logger since it is easier to read.
config.logger = [{"name": "tensorboard"}]

# We still need to explicitly set these variables.
config.trainer.exp_name = "jupyter"
config.trainer.log_dir_name = "test"
config.trainer.base_run_dir = "runs"
config.trainer.run_id = 0

In [ ]:
objs = ml.instantiate_config(config)

# Unpacking the different components.
model = objs.model
task = objs.task
optimizer = objs.optimizer
lr_scheduler = objs.lr_scheduler
trainer = objs.trainer

from tensorboard import notebook

# Show Tensorboard inside the notebook.
notebook.display(port=int(os.environ['TENSORBOARD_PORT']))

# Runs the training loop.
trainer.train(model, task, optimizer, lr_scheduler)

## Building a planner

This example demonstrates how to use the trained model to build a planner.

In [ ]:
from usa.planners.clip_sdf import GradientPlanner

# Builds the planner from the model and task. The planner
# hyperparameters can be configured as needed.
planner = GradientPlanner(
    dataset=task._dataset(),
    model=model,
    task=task,
    device=task._device,
)

In [ ]:
# Generate waypoints between two explicit points.
waypoints = planner.plan(start_xy=(0, 0), end_xy=(1, 1))

# Generates waypoints from a start location to a semantic target.
waypoints = planner.plan(start_xy=(0, 0), end_goal="The chair")